In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mytools
import mysecure

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 5]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
pwd = 'G:\\1_Data1\\91_BaikalJul2022\\'
saveImgPath = 'G:\\1_Data1\\91_BaikalJul2022\\'
saveDataPath = 'G:\\1_Data1\\91_BaikalJul2022\\'

In [ ]:
df = pd.read_csv(pwd+'origin.dat', sep='\t', parse_dates=['DateTime'], dayfirst=True, na_values='--', decimal=',', skiprows=[1,2], header=[0])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df.sort_values(by='DateTime', inplace=True)
df.reset_index(inplace=True, drop=True)
df

In [ ]:
### remove service variables
df.drop([x for x in df.columns if x.lower().endswith('err')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('max')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().endswith('min')], axis='columns', inplace=True, errors='ignore')
df.drop([x for x in df.columns if x.lower().startswith('pump')], axis='columns', inplace=True, errors='ignore')
df.drop(['HumidityAir', ], axis='columns', inplace=True, errors='ignore')


In [ ]:
df.shape, df.columns

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(7, 1)

axs[0].set_title('vCO2, ppm')
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')
axs[0].set_ylim(0, 1000)

axs[1].set_title('vCH4, ppm')
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')

axs[2].set_title('Temp')
axs[2].plot(df.DateTime, df['TempEqu1'], 'r-')
axs[2].plot(df.DateTime, df['TempEqu2'], 'g-')
axs[2].plot(df.DateTime, df['TempEqu3'], 'b-')
axs[2].plot(df.DateTime, df['TempAir'], 'c-')

axs[3].set_title('WaterFlow')
axs[3].plot(df.DateTime, df['WaterFlowEqu1'], 'r-')
axs[3].plot(df.DateTime, df['WaterFlowEqu2'], 'g-')
axs[3].plot(df.DateTime, df['WaterFlowEqu3'], 'b-')

axs[4].set_title('Press')
axs[4].plot(df.DateTime, df['PressAir'], 'c-')
axs[5].set_title('Solar')
axs[5].plot(df.DateTime, df['LightLX'], 'w-')
axs[6].set_title('fluo, r.u.')
axs[6].plot(df.DateTime, df['FluoKfaBlu'], 'b.')
axs[6].plot(df.DateTime, df['FluoKfaGrn'], 'g.')
axs[6].plot(df.DateTime, df['FluoKfaRed'], 'r.')



### Correction data

In [ ]:
## gether/apply specific variables
### data corrections
# df['vCO2'] = df['vCO2'] * scheme.CO2PicarroScaut_cor[0] + scheme.CO2PicarroScaut_cor[1]
# df['vCH4'] = df['vCH4'] * scheme.CH4PicarroScaut_cor[0] + scheme.CH4PicarroScaut_cor[1]
# df['vCO2'] = df['CO2_dry']
# df['vCH4'] = df['CH4_dry']
df['LightLX'] = df['LightLX'] * scheme.solar_cor[0] + scheme.solar_cor[1]
df['LightUV'] = df['LightUV'] * scheme.solar_uv_cor[0] + scheme.solar_uv_cor[1]
df['PressAir'] = bfunc.pressConvert(df['PressAir'])
df['PressAir'] = df['PressAir'] * scheme.press_cor[0] + scheme.press_cor[1]

### servise data corrections
df['TempEqu1'] = df['TempEqu1'] * scheme.tempEqu1_cor[0] + scheme.tempEqu1_cor[1]
df['TempEqu2'] = df['TempEqu2'] * scheme.tempEqu2_cor[0] + scheme.tempEqu2_cor[1]
df['TempEqu3'] = df['TempEqu3'] * scheme.tempEqu3_cor[0] + scheme.tempEqu3_cor[1]

df['AirFlow'] = df['AirFlow'] * scheme.airflow_cor[0] + scheme.airflow_cor[1]

df['WaterFlowEqu1'] = df['WaterFlowEqu1'] * scheme.waterflowEqu1_cor[0] + scheme.waterflowEqu1_cor[1]
df['WaterFlowEqu2'] = df['WaterFlowEqu2'] * scheme.waterflowEqu2_cor[0] + scheme.waterflowEqu2_cor[1]
df['WaterFlowEqu3'] = df['WaterFlowEqu3'] * scheme.waterflowEqu3_cor[0] + scheme.waterflowEqu3_cor[1]

###  Remove bad data   
List of accidental cases to filter data (see notes)

In [ ]:
filter_list = [{'date_start': '08.07.2022 22:38', 'date_stop': '09.07.2022 06:30', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
               {'date_start': '09.07.2022 09:26', 'date_stop': '09.07.2022 12:12', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
               {'date_start': '10.07.2022 03:53', 'date_stop': '10.07.2022 07:52', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
               {'date_start': '11.07.2022 23:55', 'date_stop': '12.07.2022 07:10', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
               {'date_start': '13.07.2022 17:44', 'date_stop': '14.07.2022 00:17', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
               {'date_start': '14.07.2022 16:55', 'date_stop': '31.12.2022 23:59', 'cols': ['chm_fail'], 'fill_with': 1},   # chamber is disfunc
              ]

df['chm_fail'] = 0
for cycle in filter_list:
    for col in cycle['cols']:
        date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
        date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
cols = ['FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu']
for col in cols:
    df.loc[df[col] == 0, col] = np.NaN
cols = ['WaterFlowEqu1', 'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4', 'AirFlow']
for col in cols:
    df.loc[df[col] < 0, col] = np.NaN

### Separate data

In [ ]:
v_state_list = list(scheme.CHANNEL_COLS.values())

chnl = 'Channel'
if not chnl in df.columns:
    df[chnl] = 0
if 'V1_state' in df.columns:
    df.loc[df.V1_state == 1, chnl] = 1
    df.loc[df.V2_state == 1, chnl] = 2
    df.loc[df.V3_state == 1, chnl] = 3
    df.loc[df.V4_state == 1, chnl] = 4
    df.loc[df.V5_state == 1, chnl] = 5
    df.loc[df.V6_state == 1, chnl] = 6
df.drop(v_state_list, inplace=True, errors='ignore')

### Flux calc

In [ ]:
### find the moments where flux is valid to calculate
df['flag'] = (df['Channel'] == 5)
df['flag'] &= (df['chm_fail'] == 0)
df['fluxEvent'] = (df['flag']==0) & (df['flag'].shift(-1)!=0)
df['fluxCount'] = df['fluxEvent'].cumsum() * df['flag']
df.tail(990).head(10)

In [ ]:
ch_v = 'flag'
df['cCO2chm'] = df['vCO2'][(df[chnl] == 5) & (df[ch_v] == 1)] * df['PressAir'] * bfunc.getDensity(temp=0, press=1, gas='CO2') / 1000  ## mg/l
df['cCH4chm'] = df['vCH4'][(df[chnl] == 5) & (df[ch_v] == 1)] * df['PressAir'] * bfunc.getDensity(temp=0, press=1, gas='CH4')         ## ug/l
df['DateSec'] = df['DateTime'].astype('int64')//10**9
for col in ['cCO2chm', 'cCH4chm']:
    df[col] = df[col].rolling(3).mean()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', 'cCO2chm', 'ro', linewidth=1, alpha=0.2, data=df )
ax.plot('DateTime', 'flag', 'c-', linewidth=1, alpha=0.2, data=df)
ax.set_xlim(pd.to_datetime('08.07.2022 20:00', dayfirst=True), pd.to_datetime('09.07.2022 10:00', dayfirst=True))
ax.set_ylim(0.6, 1.01)

In [ ]:
deep = 1
df['CO2flux'] = (df['cCO2chm'].shift(-deep) - df['cCO2chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320
df['CH4flux'] = (df['cCH4chm'].shift(-deep) - df['cCH4chm'].shift(deep)) / (df['DateSec'].shift(-deep) - df['DateSec'].shift(deep)) * 3600 / 0.81 * 320

In [ ]:
def get_mean_flux_1(dfff):
    dfff = dfff.tail(30)
    return dfff.quantile(q=0.4, interpolation='midpoint')
'''    dfff.sort_values(by='CO2flux')
    i_begin = int(len(dfff)/2) - s
    i_end = int(len(dfff)/2) + s
    
    dfff.loc[inl(len)]
   ''' 
    

dff = df.loc[:, ['DateSec', 'CO2flux', 'CH4flux', 'fluxCount']]
dff.describe()

In [ ]:
dff = dff.groupby(by='fluxCount').apply(get_mean_flux_1)
dff['DateSec'] = dff['DateSec'].astype('int')
dff

In [ ]:
df = pd.merge(left=df, right=dff, how='left', left_on='fluxCount', right_index=True, suffixes=(None, '_1'))

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot('DateTime', df['flag']*100, 'c-', linewidth=1, alpha=0.8, data=df)
ax.plot('DateTime', 'CO2flux', 'ro', linewidth=1, alpha=0.1, data=df )
ax.plot('DateTime', 'CO2flux_1', 'bo', linewidth=1, alpha=0.9, data=df )

# ax.set_xlim(pd.to_datetime('08.07.2022 0:00', dayfirst=True), pd.to_datetime('09.07.2022 0:00', dayfirst=True))
ax.set_ylim(-200, 200)
#ax.set_xlim(1656960000, 1657823000)

In [ ]:
df['reff'] = 1
df = df.resample('20S', on='DateTime').mean()
df[chnl].fillna(method='ffill', inplace=True)
ch_v = 'Chn_valid'
df[ch_v] = 1
for deep in range(1, 7):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0
df.dropna(axis='index', subset=['reff'], inplace=True)

In [ ]:
df['DateTime'] = df.index
df

In [ ]:
df['vCO2air'] = df['vCO2'][(df[chnl] == 6) & (df[ch_v] == 1)]
df['vCH4air'] = df['vCH4'][(df[chnl] == 6) & (df[ch_v] == 1)]
for col in ['vCO2air', 'vCH4air',]:
    df[col] = df[col].rolling(3).mean()

df['vCO2air'].interpolate(method='values', inplace=True)
df['vCH4air'].interpolate(method='values', inplace=True)  ## `time` method mb better
df['pCO2air'] = df['vCO2air'] * df['PressAir']  # mkatm
df['pCH4air'] = df['vCH4air'] * df['PressAir']  # mkatm

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(2, 1)

axs[0].set_title('CO2 air, ppm')
axs[0].set_ylim(300, 550)
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')
axs[0].plot(df.DateTime, df['vCO2air'], 'r-')

axs[1].set_title('CH4 air, ppm')
axs[1].set_ylim(1.75, 2.5)
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].plot(df.DateTime, df['vCH4air'], 'b-')
fig.savefig(saveImgPath+'cAir_vs_time.png', transparent=False)

In [ ]:
df.columns

## RECOVERY !!!

In [ ]:
ch_v = 'Chn_valid'
df[ch_v] = 1
for deep in range(1, 15):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64')//10**9
df['dTSec'] = df['DateSec'] - df['DateSec'].shift(1)
dt = df['dTSec'] / 60   ## delta time, min
ch_v = 'Chn_valid'

#### recovery `CO2/CH4`, channel `1` (bottom jul 2022)

In [ ]:
equ_vol = scheme.equ_walltube_param['equ_vol']  # equivalent equ volume, l
equ_cap = scheme.equ_walltube_param['equ_cap']  # equilibrator capacity
wtr_flow_min = scheme.equ_walltube_param['water_flow_min'] 

In [ ]:
t_wtr = df['TempEqu1']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']
air_flow = df['AirFlow']

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2bot'] = cGasWtr * 1000     # mg/l
df['pCO2bot'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 1) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4bot'] = cGasWtr * 1000000000    # ng/l
df['pCH4bot'] = cGasWtr * 1000000 / solubility  # mkatm

#### recovery `CO2/CH4`, channel `2` (surface in jul 2022)

In [ ]:
t_wtr = df['TempEqu2']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu2']

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 2) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2sur'] = cGasWtr * 1000     # mg/l
df['pCO2sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 2) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(df['TempEqu1'], 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4sur'] = cGasWtr * 1000000000    # ng/l
df['pCH4sur'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
## rolling average is applied inplace. Be cafelly, run this cell **ONCE**

for col in ['cCO2sur', 'cCO2bot', 'pCO2sur', 'pCO2bot', 'cCH4sur', 'pCH4sur', 'cCH4bot', 'pCH4bot']:
    df.loc[df[col] == np.inf , col] = np.nan 
    df.loc[df[col] == -np.inf , col] = np.nan           
    
    df.loc[df[col] <= 0 , col] = np.nan
    df[col] = df[col].rolling(4, center=True).mean()

df['cCH4bot'].describe()

In [ ]:
df['pCO2'] = df['vCO2'] * df['PressAir']

plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
## ax.set_xlim(pd.to_datetime('05.06.2023 18:00:00', dayfirst=True), pd.to_datetime('05.06.2023 19:00:00', dayfirst=True))
ax.set_title('CO2 water, mkatm')
# ax.set_ylim(0, 800)
ax.plot(df.DateTime, df['pCO2'], '-', c='silver')
ax.plot(df.DateTime, df['pCO2sur'], 'g-')
ax.plot(df.DateTime, df['pCO2bot'], 'r-')
ax.plot(df.DateTime, df['pCO2air'], 'c-')
fig.savefig(saveImgPath+'pCO2wtr_vs_time.png')

In [ ]:
count_recent = 30000
df['pCH4'] = df['vCH4'] * df['PressAir']
plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
ax.set_xlim(pd.to_datetime('07.07.2022 10:00:00', dayfirst=True), pd.to_datetime('09.07.2022 19:00:00', dayfirst=True))
# ax.set_ylim(0, 20)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='silver')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4air'], 'c.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4bot'], 'r.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4sur'], 'g.')
fig.savefig(saveImgPath+'pCH4wtr_vs_time.png')

In [ ]:
df.columns

In [ ]:
df['hour'] = df['DateTime'].dt.hour + df['DateTime'].dt.minute / 60.0 + + df['DateTime'].dt.second / 3600.0
df['hour']

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1)
ax.plot('hour', 'CO2flux', 'ro', linewidth=1, alpha=0.1, data=df )
ax.plot('hour', 'CO2flux_1', 'bo', linewidth=1, alpha=0.9, data=df )
ax.set_ylim(-200, 200)

In [ ]:
cols = ['hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'vCO2', 'vCH4', 'vH2O', 'AirFlow', 'Channel',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu', 'WaterFlowEqu1',
        'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4',
        'TempEqu1', 'TempEqu2', 'TempEqu3', 'cCO2chm', 'cCH4chm',
        'CO2flux', 'CH4flux', 'vCO2air', 'vCH4air',
        'CO2flux_1', 'CH4flux_1',
        'pCO2air', 'pCH4air', 'cCO2sur', 'pCO2sur',
        'cCH4sur', 'pCH4sur', 'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',]
df.to_csv(f'{pwd}origin_pd.txt',
          columns=cols,
          sep='\t',
         )

In [ ]:
hourly = df.resample('60T', on='DateTime', offset='30T').mean()
hourly.index = hourly.index + pd.to_timedelta('30T')

cols = ['hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'vCO2', 'vCH4', 'vH2O',
        'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu', 'WaterFlowEqu1',
        'TempEqu1', 'TempEqu2', 'TempEqu3', 'cCO2chm', 'cCH4chm',
        'CO2flux', 'CH4flux', 'vCO2air', 'vCH4air',
        'CO2flux_1', 'CH4flux_1',
        'pCO2air', 'pCH4air', 'cCO2sur', 'pCO2sur',
        'cCH4sur', 'pCH4sur', 'cCO2bot', 'pCO2bot', 'cCH4bot', 'pCH4bot',]
hourly.to_csv(f'{saveDataPath}hourly.txt',
              columns=cols,
              sep='\t',
             )